In [3]:
import sys
sys.path.append(r'E:\0code')

In [56]:
import pyml
import pandas as pd
import numpy as np
from pyml.neighbors.regression import KNeighborsRegressor

# 读取数据文件

In [48]:
train = pd.read_excel('./data/train.xlsx')
test = pd.read_excel('./data/testStudent.xlsx')

In [11]:
train.head()

,Additional_Number_of_Scoring,Average_Score,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Tags,Reviewer_Score
0,255,8.1,57,1290,10,1,"[' Family with young children ', ' Standard Qu...",6.3
1,211,8.6,0,2923,19,12,"[' Leisure trip ', ' Family with young childre...",10.0
2,189,9.2,27,781,29,24,"[' Business trip ', ' Couple ', ' Superior Roo...",10.0
3,1258,9.4,20,4204,31,3,"[' Leisure trip ', ' Couple ', ' Deluxe King R...",9.2
4,289,8.8,0,1519,3,9,"[' Leisure trip ', ' Solo traveler ', ' Deluxe...",10.0


In [14]:
train.dtypes

Additional_Number_of_Scoring                    int64
Average_Score                                 float64
Review_Total_Negative_Word_Counts               int64
Total_Number_of_Reviews                         int64
Review_Total_Positive_Word_Counts               int64
Total_Number_of_Reviews_Reviewer_Has_Given      int64
Tags                                           object
Reviewer_Score                                float64
dtype: object

In [27]:
train.Tags[0].strip("[]").replace("'",'').split(',')

[' Family with young children ', '  Standard Queen Room ', '  Stayed 1 night ']

In [ ]:
def preprocessing_data(df):
    pass

# 定义特征构建函数

https://github.com/ShawnyXiao/2017-CCF-BDCI-Enterprise 该师兄的分析我觉得蛮有道理的


1. 基础特征
2. 偏离值特征
3. 交叉特征
4. 想象力特征hhh

观察数据，发现
1. 通过比值构建新特征：构造积极、消极、评论员评论占比特征

In [51]:
def get_proportion_feature(df):
    df = df.copy()
    
    base_features = ['Total_Number_of_Reviews']
    gap_features = ['Review_Total_Negative_Word_Counts', 'Review_Total_Positive_Word_Counts','Total_Number_of_Reviews_Reviewer_Has_Given']
    for base_feature in base_features:
        for gap_feature in gap_features:
            df[gap_feature+'_radio_'+base_feature] = df[gap_feature]/df[base_feature] * 10
            # 数字太小了，乘上一个10
            df = df.drop(gap_feature, axis=1)
    return df

# 调用函数 构建特征

In [52]:
train_feat = get_proportion_feature(train)
test_feat = get_proportion_feature(test)

In [53]:
train_feat.columns

Index(['Additional_Number_of_Scoring', 'Average_Score',
       'Total_Number_of_Reviews', 'Tags', 'Reviewer_Score',
       'Review_Total_Negative_Word_Counts_radio_Total_Number_of_Reviews',
       'Review_Total_Positive_Word_Counts_radio_Total_Number_of_Reviews',
       'Total_Number_of_Reviews_Reviewer_Has_Given_radio_Total_Number_of_Reviews'],
      dtype='object')

# 构建训练集与测试集

In [54]:
train_feat = train_feat.drop(labels=['Tags'], axis=1)
test_feat = test_feat.drop(labels=['Tags'], axis=1)

In [60]:
train_label = train_feat['Reviewer_Score']
train_feat = train_feat.drop('Reviewer_Score', axis=1)

In [61]:
train_feat.columns

Index(['Additional_Number_of_Scoring', 'Average_Score',
       'Total_Number_of_Reviews',
       'Review_Total_Negative_Word_Counts_radio_Total_Number_of_Reviews',
       'Review_Total_Positive_Word_Counts_radio_Total_Number_of_Reviews',
       'Total_Number_of_Reviews_Reviewer_Has_Given_radio_Total_Number_of_Reviews'],
      dtype='object')

In [62]:
test_feat.columns

Index(['Additional_Number_of_Scoring', 'Average_Score',
       'Total_Number_of_Reviews',
       'Review_Total_Negative_Word_Counts_radio_Total_Number_of_Reviews',
       'Review_Total_Positive_Word_Counts_radio_Total_Number_of_Reviews',
       'Total_Number_of_Reviews_Reviewer_Has_Given_radio_Total_Number_of_Reviews'],
      dtype='object')

# 导入模型与训练

In [57]:
kreg = KNeighborsRegressor()

In [65]:
kreg.fit(train_feat.values, train_label.values.reshape(-1,1))

In [ ]:
y_pred = kreg.predict(test_feat.values)

# 结果文件的写出

In [ ]:
sub = pd.DataFrame(y_pred)
sub.to_csv('./results/'+ str(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")) + ".csv", index=0, header=None, index_label=None)